In [0]:
%run ../config/init_config


In [0]:
from pyspark.sql.functions import current_timestamp

query = (spark.readStream
              .format(file_format)
              .option("cloudFiles.format", file_format)
              .option("header", "true")
              .schema(source_schema)
              .load(source_path)
              .withColumn("processed_time", current_timestamp())
              .writeStream
              .option("checkpointLocation", bronze_checkpoint_path)
              .option("mergeSchema", "true")
              .partitionBy("source_type")
              .trigger(availableNow=True)
              .table("bronze_raw")
)

query.awaitTermination()

display(spark.read.table("bronze_raw"))

In [0]:
%sql DESCRIBE EXTENDED bronze_raw

INSERT INTO bronze_raw
SELECT * FROM VALUES
('vehicles', "{'vehicle_id': 'VH001', 'brand': 'Toyota', 'model': 'Corolla', 'price': 30, 'country': 'France'}", current_timestamp())

In [0]:
display(spark.read.table("bronze_raw"))